In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) 
[Clang 11.1.0 ]
Pandas 1.3.4
Scikit-Learn 1.0.1
GPU is available


In [2]:
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
stop=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
# import gensim
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from tensorflow.keras.initializers import Constant
from sklearn.model_selection import train_test_split
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sqlalchemy import create_engine
import pymysql

sqlEngine = create_engine('mysql+pymysql://root:@127.0.0.1/nlp_with_disaster_tweets', pool_recycle=3600)
dbConnection = sqlEngine.connect()

### Read combined data

In [6]:
df_combined = pd.read_sql_query(
'''
select *
from transformed_combination
'''
,dbConnection)

df_train = df_combined[df_combined['source_file']=='train']
df_test = df_combined[df_combined['source_file']=='test']

In [7]:
def create_corpus(df: pd.DataFrame) ->list:
    '''
    The function is to convert column "text" into a list of words at each row

    input:
    df: dataframe where text whose words are splited

    output:
    return a list of words for each row
    '''
    corpus = []
    
    for tweet in tqdm(df['text']):
        words = [word.lower() for word in word_tokenize(tweet) if (word.isalpha() == 1) & (word not in stop)]

        corpus.append(words)

    return corpus


In [8]:
corpus = create_corpus(df_combined)

100%|██████████| 10876/10876 [00:01<00:00, 10795.83it/s]


#### Convert glove.6B.100d.txt to a dictionary

In [9]:
embedding_dict={}
with open('../../Project_data/nlp_with_disaster_tweets/glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

<font color='red'> ** Need to understand the following cells ** </font>

<a id="Summary"></a>
<div class="alert alert-block alert-success">
    <font color = 'black'>
        <h2>Tokenization</h2>
        <h3>What is Tokenization?</h3>
    </font>
</div>

In [10]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
# Updates internal vocabulary based on a list of texts
# fit_on_text: the function is to return the index of each word
# by following instructions:
# 1. get frequency for each word for the whole list (e.g. corpus)
# 2. sort the word by frequency of words by descending
# 3. create index by the order of the words' frequency from step 2.
# For example, the most frequency of a word is 1 
tokenizer_obj.fit_on_texts(corpus)
# texts_to_sequences: use the index from the function fit_on_text
# to map each word to index
sequences=tokenizer_obj.texts_to_sequences(corpus)
# make each list extend to max_len. If the number of items is lower than max_len,
# append 0 ; else truncating the bottom items until the length of the list same as max_len
tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

The following scripts is a simple versioin of function fit_on_text<br>
The original one is in the following linke <https://github.com/keras-team/keras-preprocessing/blob/1.1.2/keras_preprocessing/text.py#L199-L251>

In [11]:
# document_count = 0
# word_counts, word_docs = {}, {}

# for text in corpus:
#     document_count+=1
#     text = [text_elem.lower() for text_elem in text]
#     seq = text 

#     for w in seq:
#         if w in word_counts:
#             word_counts[w] += 1
#         else:
#             word_counts[w] = 1

#     for w in set(seq):
#         if w in word_docs:
#             word_docs[w] += 1
#         else:
#             word_docs[w] = 1
# wcounts = list(word_counts.items())
# wcounts.sort(key=lambda x: x[1], reverse=True)

# sorted_voc = []
# sorted_voc.extend(wc[0] for wc in wcounts)

# word_index = dict(
#             zip(sorted_voc, list(range(1, len(sorted_voc) + 1))))

# index_word = {c: w for w, c in word_index.items()}
# index_docs={}
# for w, c in list(word_docs.items()):
#     index_docs[word_index[w]] = c

In [11]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 18736


In [13]:
num_words=len(word_index)+1
# create a new array with num_words sub-list and 100 items in the sub-list filled with zero
embedding_matrix=np.zeros((num_words,100))

# the loop is to fill glove.6B.100d.txt for each word
for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

100%|██████████| 18736/18736 [00:00<00:00, 830342.87it/s]


<font color='red'> to be investigated!!!!</font>

In [14]:
model=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           1873700   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 50, 100)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 1,916,005
Trainable params: 42,305
Non-trainable params: 1,873,700
_________________________________________________________________


In [16]:
train=tweet_pad[:df_train.shape[0]]
test=tweet_pad[df_train.shape[0]:]

In [17]:
X_train,X_test,y_train,y_test=train_test_split(train,df_train['target'].values,test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6471, 50)
Shape of Validation  (1142, 50)


In [22]:
history=model.fit(X_train,y_train,batch_size=4,epochs=50,validation_data=(X_test,y_test),verbose=2)

Epoch 1/50
1618/1618 - 36s - loss: 0.4681 - accuracy: 0.7926 - val_loss: 0.4530 - val_accuracy: 0.8056
Epoch 2/50
1618/1618 - 36s - loss: 0.4704 - accuracy: 0.7866 - val_loss: 0.4466 - val_accuracy: 0.8126
Epoch 3/50
1618/1618 - 36s - loss: 0.4725 - accuracy: 0.7860 - val_loss: 0.4516 - val_accuracy: 0.8012
Epoch 4/50
1618/1618 - 36s - loss: 0.4680 - accuracy: 0.7889 - val_loss: 0.4485 - val_accuracy: 0.8039
Epoch 5/50
1618/1618 - 36s - loss: 0.4631 - accuracy: 0.7954 - val_loss: 0.4542 - val_accuracy: 0.8030
Epoch 6/50
1618/1618 - 36s - loss: 0.4629 - accuracy: 0.7972 - val_loss: 0.4432 - val_accuracy: 0.8126
Epoch 7/50
1618/1618 - 36s - loss: 0.4623 - accuracy: 0.7912 - val_loss: 0.4413 - val_accuracy: 0.8135
Epoch 8/50
1618/1618 - 36s - loss: 0.4572 - accuracy: 0.7968 - val_loss: 0.4550 - val_accuracy: 0.8047
Epoch 9/50
1618/1618 - 36s - loss: 0.4597 - accuracy: 0.7918 - val_loss: 0.4483 - val_accuracy: 0.8065
Epoch 10/50
1618/1618 - 36s - loss: 0.4587 - accuracy: 0.8013 - val_loss:

In [23]:
sample_sub = pd.read_csv('../../Project_data/nlp_with_disaster_tweets/sample_submission.csv')

In [21]:
y_pre=model.predict(test)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('../../Project_data/nlp_with_disaster_tweets/submit/submission.csv',index=False)